In [1]:
import time,sys,cv2,datetime,os,math
import tensorflow as tf
import array as arr
import numpy as np
import scipy.io as sio
from PIL import Image
import pandas as pd
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as image
%matplotlib inline
%load_ext tensorboard

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPool2D, GlobalAveragePooling2D, BatchNormalization, MaxPooling2D, Conv2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import decode_predictions
from sklearn.metrics import classification_report

from imgaug import augmenters as iaa
from tensorflow.keras.applications.xception import Xception, preprocess_input

In [2]:
image_path = '/project/dogs/Images'
num_of_categories = 120
image_size = 300

In [3]:
breed_list = sorted(os.listdir(image_path))
num_classes = len(breed_list)
print("{} breeds".format(num_classes))

120 breeds


In [4]:
label_maps = {}
label_maps_rev = {}
for i, v in enumerate(breed_list):
    label_maps.update({v: i})
    label_maps_rev.update({i : v})

In [5]:
def paths_and_labels():
    pic_paths1 = list()
    num_labels1 = list()
    pic_paths2 = list()
    num_labels2 = list()
    mat_file1 = sio.loadmat('/project/dogs/lists/train_list.mat')
    train_list = mat_file1['file_list']
    mat_file2 = sio.loadmat('/project/dogs/lists/test_list.mat')
    test_list = mat_file2['file_list']

    for item in train_list:
        splited = np.char.split(item[0],sep = '/')
        dirs = "/project/dogs/Images/" + splited[0][0] +"/" + splited[0][1]
        pic_paths1.append(dirs)
        y_name = splited[0][0]       
        num_labels1.append(label_maps[y_name])

    for item in test_list:
        splited = np.char.split(item[0],sep = '/')
        dirs = "/project/dogs/Images/" + splited[0][0] +"/" + splited[0][1]
        pic_paths2.append(dirs)
        y_name = splited[0][0]       
        num_labels2.append(label_maps[y_name])

    return pic_paths1, np.asarray(num_labels1), pic_paths2, np.asarray(num_labels2)                  

In [6]:
x_train, y_train, x_test, y_test = paths_and_labels()

In [8]:
len(x_train),len(y_train),len(x_test),len(y_test)

(12000, 12000, 8580, 8580)

In [9]:
class ImageGenerator(Sequence):   
    def __init__(self, paths, targets, batch_size, shape, augment=False):
        self.paths = paths
        self.targets = targets
        self.batch_size = batch_size
        self.shape = shape
        self.augment = augment
        
    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_paths = self.paths[idx * self.batch_size : (idx + 1) * self.batch_size]
        x = np.zeros((len(batch_paths), self.shape[0], self.shape[1], self.shape[2]), dtype=np.float32)
        y = np.zeros((self.batch_size))
        for i, path in enumerate(batch_paths):
            x[i] = self.__load_image(path)
        y = self.targets[idx * self.batch_size : (idx + 1) * self.batch_size]
        return x, y
    
    def __iter__(self):
        for item in (self[i] for i in range(len(self))):
            yield item
            
    def __load_image(self, path):
        image = cv2.imread(path)
        image = preprocess_input(image)
        if self.augment:
            sometimes = lambda aug: iaa.Sometimes(0.5, aug)
            seq = iaa.Sequential([
                iaa.OneOf([
                    iaa.Fliplr(0.5),
            iaa.Flipud(0.5),
                    iaa.Crop(percent=(0, 0.1)),
                    iaa.Sometimes(0.5,),
                    iaa.LinearContrast((0.75, 1.5)),
                    iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-45, 45),
                        shear=(-8, 8)
                    )
                ])
            ], random_order=True)
            image = seq.augment_image(image)
        return image

In [10]:
from sklearn.utils import shuffle
x_train,y_train = shuffle(x_train,y_train)

In [11]:
batch = 32
train_ds = ImageGenerator(x_train, y_train, batch_size=batch, shape=(image_size, image_size,3), augment=True)
test_ds = ImageGenerator(x_test, y_test, batch_size=batch, shape=(image_size, image_size,3), augment=False)

In [12]:
base_model = tf.keras.applications.xception.Xception(weights='imagenet',include_top=False, pooling='avg')#Summary of Xception Model
base_model.trainable = False

In [13]:
flat_dim = 5 * 5 * 2048

model = Sequential(base_model)
model.add(Dense(1032, activation='relu',input_dim=flat_dim))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(120, activation='softmax'))

In [14]:
learning_rate_init = 5e-05
###################
def lr_scheduler(epoch):
    epoch += 1
    if epoch == 1:
        return learning_rate_init
    
    elif epoch >= 2 and epoch <= 40:
        return (0.2*epoch**3)*math.exp(-0.45*epoch)*learning_rate_init
    
    else:
        return lr_scheduler(40-1)
stage = [i for i in range(0,25)]
learning_rate = [lr_scheduler(x) for x in stage]

In [15]:
scheduler = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience = 6, mode='max', min_delta=1, verbose=1)

In [16]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, steps_per_epoch=len(x_train)/batch, validation_data=test_ds,  
                              validation_steps=len(x_test)/batch, epochs=5, callbacks=[scheduler])

In [50]:
# model.load_weights("weights2.h5")

In [17]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.RMSprop(lr=5e-05),
              metrics=['accuracy']) 

In [18]:
base_model.trainable = True

In [19]:
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 90

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  133


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 2048)              20861480  
_________________________________________________________________
dense (Dense)                (None, 1032)              2114568   
_________________________________________________________________
dropout (Dropout)            (None, 1032)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               528896    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 120)               3

In [ ]:
history_fine = model.fit(train_ds, steps_per_epoch=len(x_train)/batch, validation_data=test_ds,  
                              validation_steps=len(x_test)/batch, epochs=total_epochs)
                        

In [28]:
model.load_weights("weights3.3_ft.h5")

In [29]:
test_loss, test_accuracy = model.evaluate(test_ds,steps=int(100))

print("Test results \n Loss:",test_loss,'\n Accuracy',test_accuracy)

100/100 [==============================] - 19s 193ms/step - loss: 0.3998 - accuracy: 0.8856
Test results 
 Loss: 0.39977753162384033 
 Accuracy 0.8856250047683716


In [23]:
size = len(x_test)/batch
predict = model.predict(test_ds,steps=size)

In [24]:
preds_cls_idx = predict.argmax(axis=-1)

In [25]:
classification_report(y_test,preds_cls_idx)

'              precision    recall  f1-score   support\n\n           0       0.84      0.83      0.83        52\n           1       0.95      0.89      0.92        85\n           2       0.96      0.93      0.95       152\n           3       0.86      0.90      0.88        49\n           4       0.84      0.88      0.86       114\n           5       0.94      0.95      0.95        88\n           6       0.95      0.98      0.96        96\n           7       0.87      0.85      0.86        72\n           8       0.81      0.78      0.79        72\n           9       0.99      0.99      0.99       139\n          10       0.90      0.95      0.92        75\n          11       0.86      0.88      0.87        95\n          12       0.94      0.86      0.90        87\n          13       0.89      0.87      0.88        71\n          14       0.85      0.85      0.85        59\n          15       0.77      0.77      0.77        53\n          16       0.76      0.74      0.75        57\n       